# <font color=#c51b8a>Mine-n-Match (MnM) Mini Tutorial:</font>
### Make sure to load all the neccessary imports everytime you open this notebook!

In [4]:
# Import functions for mining NCBI
from mine_ncbi_functions import ncbi_fetch_species, ncbi_mine_seq_data 
# Import json so we can load any existing 
import json
# Email for when we query NCBI
email = "sethfrazer@ucsb.edu"  # Replace with your email


## <font color=#c994c7>Step 1: Create a Taxonomy Database</font>

In [7]:
# Example usage:
taxa = "malacostraca"
rank = "Class"
limit = 500
report_dir = 'taxonomy_data'
out_file = "malacostraca_taxonomy"

# If verbose = True then you will see alot of information printed when you run this function
# Also, if you hover your mouse over any of MY functions then you should get a pop-up with more information on how to use them
species_data = ncbi_fetch_species(email, report_dir=report_dir, out=out_file, taxa=taxa, rank=rank, limit=limit, verbose=True)

# Print the first few entries of the dictionary as an example:
count = 0
for species, lineage in species_data.items():
    print(f"{species}: {lineage}")
    count += 1
    if count >= 5:
        break

Searching for TaxID for malacostraca...
Found TaxID: 6681
Finding direct children of malacostraca...
Found 17 direct children.
  - Child Taxon: Ingolfiellida (TaxID: 2692242)
  - Child Taxon: Thermosbaenacea (TaxID: 75402)
  - Child Taxon: Spelaeogriphacea (TaxID: 75401)
  - Child Taxon: Lophogastrida (TaxID: 75400)
  - Child Taxon: Mysida (TaxID: 75399)
  - Child Taxon: Tanaidacea (TaxID: 75398)
  - Child Taxon: Mictacea (TaxID: 75397)
  - Child Taxon: Anaspidacea (TaxID: 75396)
  - Child Taxon: Bathynellacea (TaxID: 75395)
  - Child Taxon: Amphionidacea (TaxID: 75394)
  - Child Taxon: Stomatopoda (TaxID: 75390)
  - Child Taxon: Leptostraca (TaxID: 58364)
  - Child Taxon: Isopoda (TaxID: 29979)
  - Child Taxon: Cumacea (TaxID: 27412)
  - Child Taxon: Amphipoda (TaxID: 6821)
  - Child Taxon: Euphausiacea (TaxID: 6816)
  - Child Taxon: Decapoda (TaxID: 6683)

Fetching up to 30 species for child TaxID 2692242...
[{'TaxId': '371528', 'ScientificName': 'Ingolfiella tabularis', 'OtherNames'

In [ ]:
# If you want to load an existing taxonomy file, this is how you do it
#species_data_file = './taxonomy_data/mammalia_taxonomy.json'
#with open(species_data_file, 'r') as f:
#    species_data = json.load(f)

## <font color=#c994c7>Step 2: Query NCBI For a Protein Group/Family of Interest</font>

In [8]:
# Convert all the species we found in our NCBI taxonomy search to a list.
# The species names are the KEYS in the dictionary we created during the search.
species_list = list(species_data.keys())

In [9]:
query = f"(opsin[Title] OR rhodopsin[Title] OR OPN[Title] OR rh1[Title] OR rh2[Title] OR Rh1[Title] OR Rh2[Title]) NOT partial[Title] NOT voucher[All Fields] NOT kinase[All Fields] NOT kinase-like[All Fields] NOT similar[Title] NOT homolog[Title] NOT opsin-like[Title]"

# Remember, if you hover your mouse over any of MY functions then you should get a pop-up with more information on how to use them
ncbi_query_df, query_report_dir = ncbi_mine_seq_data(email=email, job_label=f'ncbi_{taxa}_opsins', out=f'ncbi_{taxa}_opsins', species_list=species_list, taxa_dictionary=species_data, query=query)

Creating Job Directory

Saving Species Query List to Text

Starting Queries to NCBI for DNA/Protein Sequences



  0% (0 of 367) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (1 of 367) |                        | Elapsed Time: 0:00:02 ETA:   0:13:08
  0% (2 of 367) |                        | Elapsed Time: 0:00:03 ETA:   0:10:52
  0% (3 of 367) |                        | Elapsed Time: 0:00:05 ETA:   0:10:36
  1% (4 of 367) |                        | Elapsed Time: 0:00:06 ETA:   0:10:05
  1% (5 of 367) |                        | Elapsed Time: 0:00:08 ETA:   0:09:57
  1% (6 of 367) |                        | Elapsed Time: 0:00:09 ETA:   0:09:43
  1% (7 of 367) |                        | Elapsed Time: 0:00:11 ETA:   0:09:33
  2% (8 of 367) |                        | Elapsed Time: 0:00:12 ETA:   0:09:26
  2% (9 of 367) |                        | Elapsed Time: 0:00:14 ETA:   0:09:22
  2% (10 of 367) |                       | Elapsed Time: 0:00:15 ETA:   0:09:15
  2% (11 of 367) |                       | Elapsed Time: 0:00:17 ETA:   0:09:15
  3% (12 of 367) |                      

NCBI Queries Complete!
Now Extracting and Formatting Results For DataFrame...

DataFrame Formatted and Saved to CSV file for future use :)

FASTA File Saved...

Saving txt file with names of species that retrieved no results...



## <font color=#c994c7>Step 3: Extract Candidate Genes From a Trasncriptome of Interest Using Queried Data</font>

In [10]:
import subprocess  # For running external programs like blast
import datetime    # For timestamping output filenames
import os          # For interacting with the filesystem
import pandas as pd # For dataframe manipulation

from blast_matching_functions import format_blast_db, run_blast_query # Our own set of functions to format blast-dbs and run blast queries 

In [ ]:
# Set all our variables, including directories to the reference database and trascriptome data
#DB_folder = "./mnm_data/mnm_on_ncbi_mammalia_opsins_2025-05-20_17-00-42"
DB_folder = query_report_dir
QUERY_folder = "./transcriptomes/ostracod_seqData/aa"
results_folder = './blast_results/opsins'
job_name = 'opsin_trans_mining'

if not os.path.isdir(results_folder):
    os.makedirs(results_folder, exist_ok=True)
    print(f"Created results folder: {results_folder}")

# blastp parameters
evalue = "1e-5"
outfmt = "6" # Tabular format, essential for pandas import
blasttyp = 'blastp' # Ensure this matches the type of query and DB

# Define standard column names for BLAST output format 6
BLAST_COLUMN_NAMES = [
    'qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen',
    'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore'
]

print(f"DB folder: {DB_folder}")
print(f"QUERY folder: {QUERY_folder}")
print(f"Results will be saved in: {results_folder}")
print(f"Job name / Protein family: {job_name}")
print("---")

DB folder: mnm_data/mnm_on_ncbi_malacostraca_opsins_2025-05-22_11-08-00
QUERY folder: ./transcriptomes/ostracod_seqData/test
Results will be saved in: ./blast_results/opsins
Job name / Protein family: opsin_trans_mining
---


In [12]:
for db_filename in os.listdir(DB_folder):
    if db_filename.endswith('.fasta') or db_filename.endswith('.faa'):
        print(f"\nProcessing Database File: {db_filename}")
        db_path_full = os.path.join(DB_folder, db_filename)
        db_name_base = os.path.splitext(db_filename)[0].replace('.', '_') # For naming output files

        # Format DB (once per database file)
        if not format_blast_db(db_path_full):
            print(f"  Skipping database {db_filename} due to formatting error.")
            continue # Move to the next database file
        
        # This is the base path that BLAST tools will use with the -db argument
        db_blast_base_path = os.path.splitext(db_path_full)[0]

        all_best_hits_for_this_db_list = [] # Accumulate best hits for THIS DB from all query files

        for query_filename in os.listdir(QUERY_folder):
            if query_filename.lower().endswith(('.fasta', '.fa', '.faa', '.pep')):
                print(f"  Processing Query File: '{query_filename}' against DB: '{db_filename}'")
                query_path_full = os.path.join(QUERY_folder, query_filename)
                query_name_base = os.path.splitext(query_filename)[0]

                # Output file for this specific BLAST run's raw results
                if not os.path.isdir(f"{results_folder}/raw_blast_results"):
                    os.makedirs(f"{results_folder}/raw_blast_results", exist_ok=True)
                time_stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                temp_raw_blast_output_file = os.path.join(
                    f"{results_folder}/raw_blast_results/",
                    f"{job_name}_{query_name_base}_vs_{db_name_base}_{time_stamp}.tsv"
                )

                # Run BLAST and get DataFrame
                blast_results_df = run_blast_query(
                    blasttyp, query_path_full, db_blast_base_path,
                    temp_raw_blast_output_file, evalue, outfmt, BLAST_COLUMN_NAMES
                )

                if blast_results_df is not None and not blast_results_df.empty:
                    # Ensure 'pident' is numeric for correct identification of maximum
                    blast_results_df['pident'] = pd.to_numeric(blast_results_df['pident'], errors='coerce')
                    blast_results_df.dropna(subset=['pident'], inplace=True) # Remove rows if pident wasn't numeric

                    if not blast_results_df.empty:
                        # Filter to keep only the hit with the highest 'pident' for each 'qseqid'
                        # Using .copy() to avoid SettingWithCopyWarning
                        idx = blast_results_df.groupby('qseqid', group_keys=False)['pident'].idxmax()
                        best_hits_df = blast_results_df.loc[idx].copy()
                        
                        best_hits_df['source_transcriptome_file'] = query_filename # Add source file info
                        all_best_hits_for_this_db_list.append(best_hits_df)
                        print(f"    Found {len(best_hits_df)} best hits from '{query_filename}'.")
                    else:
                        print(f"    No valid numeric 'pident' values in results for '{query_filename}' against '{db_filename}'.")
                    
                    # Optional: Remove the temporary raw BLAST output file if it's no longer needed
                    # if os.path.exists(temp_raw_blast_output_file):
                    #     os.remove(temp_raw_blast_output_file)
                    #     print(f"    Removed temporary file: {temp_raw_blast_output_file}")

                else:
                    print(f"    No results or error processing BLAST output for '{query_filename}' against '{db_filename}'.")
                    # Optional: remove empty temp file if created
                    # if os.path.exists(temp_raw_blast_output_file) and os.path.getsize(temp_raw_blast_output_file) == 0:
                    #    os.remove(temp_raw_blast_output_file)


        # After processing all query files for the current database
        if all_best_hits_for_this_db_list:
            final_compiled_df_for_db = pd.concat(all_best_hits_for_this_db_list, ignore_index=True)
            
            # Final compiled CSV filename: jobName_dbName_bestHits.csv
            compiled_csv_filename = os.path.join(results_folder, f"{job_name}_{db_name_base}_best_hits_compiled_{time_stamp}.csv")
            final_compiled_df_for_db.to_csv(compiled_csv_filename, index=False)
            
            print(f"\n  Successfully compiled {len(final_compiled_df_for_db)} total best hits for database '{db_filename}'.")
            print(f"  Results saved to: {compiled_csv_filename}")
        else:
            print(f"\n  No best hits found for any query files against database '{db_filename}'.")
        print("---")



Processing Database File: mined_ncbi_malacostraca_opsins_seqs.fasta
Formatting BLAST DB for: mnm_data/mnm_on_ncbi_malacostraca_opsins_2025-05-22_11-08-00\mined_ncbi_malacostraca_opsins_seqs.fasta
  Database 'mnm_data/mnm_on_ncbi_malacostraca_opsins_2025-05-22_11-08-00\mined_ncbi_malacostraca_opsins_seqs' formatted successfully.
  Processing Query File: 'Australia_Alternochelata_lizardensis__AltLiz.fa' against DB: 'mined_ncbi_malacostraca_opsins_seqs.fasta'
    Found 5 best hits from 'Australia_Alternochelata_lizardensis__AltLiz.fa'.
  Processing Query File: 'Australia_Chelicopia_lorica__Chlorica.fa' against DB: 'mined_ncbi_malacostraca_opsins_seqs.fasta'
    Found 15 best hits from 'Australia_Chelicopia_lorica__Chlorica.fa'.
  Processing Query File: 'Australia_CYPRIDINIDAE__FatWhiteEye.fa' against DB: 'mined_ncbi_malacostraca_opsins_seqs.fasta'
    Found 20 best hits from 'Australia_CYPRIDINIDAE__FatWhiteEye.fa'.

  Successfully compiled 40 total best hits for database 'mined_ncbi_mal